In [4]:
#%pip install torch

#%pip install lightning


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/808.5 kB ? eta -:--:--
   --------------------------------------- 808.5/808.5 kB 11.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/866.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/866.2 kB ? eta -:--:--
   ---------------------------------------- 866.2/866.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/812.3 kB ? eta -:--:--
   --------------------------------------- 812.3/812.3 kB 17.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch

#Weights and Biases
import torch.nn as nn

#Activition Function
import torch.nn.functional as F

#Fit the neural network to the data (find the optimal values - similar to SGD)
from torch.optim import Adam

import lightning as L

from torch.utils.data import TensorDataset, DataLoader 


C:\Users\dangn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
class LSTMbyHand(L.LightningModule):
    def __init__(self):
        #Create and initialize weight and bias tensors
        super().__init__()

        #Using normal distribution near 0 
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)

        #weights
        self.wlr1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wlr2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        #bias
        self.blr1 = nn.Parameter(torch.tensor(0.), requires_grad = True)


        self.wpr1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wpr2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bpr1 = nn.Parameter(torch.tensor(0.), requires_grad = True)


        self.wp1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wp2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bp1 = nn.Parameter(torch.tensor(0.), requires_grad = True)
        

        self.wo1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.wo2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad = True)
        self.bo1 = nn.Parameter(torch.tensor(0.), requires_grad = True)

    def lstm_unit(self, input_value, long_memory, short_memory):
        #LSTM Math

        #First Stage
        long_remember_percent = torch.sigmoid((short_memory * self.wlr1) + (input_value * self.wlr2) + self.blr1)


        #Second Stage 
        potential_remember_percent = torch.sigmoid((short_memory * self.wpr1) + (input_value * self.wpr2) + self.bpr1)

        potential_memory = torch.tanh((short_memory * self.wp1) + (input_value * self.wp2) + self.bp1)
        
        #First scaling + Second scaling in the long term memory
        updated_long_memory = ((long_memory * long_remember_percent) + (potential_remember_percent * potential_memory))
        
        #Third scaling
        output_percent = torch.sigmoid((short_memory * self.wo1) + (input_value * self.wo2) + self.bo1)

        updated_short_memory = torch.tanh(updated_long_memory) * output_percent

        return([updated_long_memory, updated_short_memory])
    
    
        

    def forward(self, input):
        #Make a forward pass through unrolled LSTM
        long_memory = 0
        short_memory = 0
        day1 = input[0]
        day2 = input[1]
        day3 = input[2]
        day4 = input[3]

        long_memory, short_memory = self.lstm_unit(day1, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day2, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day3, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day4, long_memory, short_memory)

        return short_memory



    def configure_optimizers(self):
        #Configure Adam optimizer
        return Adam(self.parameters())

    def training_step(self, batch, batch_idx):
        #Calculate loss and log training progress
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i  - label_i)**2

        self.log("train loss", loss)

        if (label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)
        
        return loss





In [15]:
model = LSTMbyHand()

print("Company A: Observed = 0, Predicted =", model(torch.tensor([0., 0.5, 0.25,1.])).detach())

Company A: Observed = 0, Predicted = tensor(0.0687)


In [18]:
#Training Data
inputs = torch.tensor([[0.,0.5,0.25,1.], [1., 0.5, 0.25, 1.]])

#Labels or Actuals
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [ ]:

#Find the optimal parameters, which mean we want to find the optimal weight
trainer = L.Trainer(max_epochs=2000)
